## Imports

In [12]:
import torch
import torch.nn as nn

## Tensorboard writer

In [13]:
from torch.utils.tensorboard import SummaryWriter

## Initialize Tensorboard

In [14]:
writer = SummaryWriter("runs/mnist_trial")

In [26]:
class Neural_Network(nn.Module):
    def __init__(self,input_size , hidden_size , num_classes):
        super(Neural_Network,self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)
    
    def forward(self,x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        ## if applying nn.BCEloss then involve sigmoid at last
        y_pred = nn.Sigmoid(out)
        return y_pred

In [27]:
model = Neural_Network(28*28,5,1)

In [28]:
criterion = nn.BCELoss()

## Load MNIST dataset

In [29]:
import torchvision
from torchvision import transforms

In [30]:
train_dataset = torchvision.datasets.MNIST(root ="./data",download = True, train = True,transform = transforms.ToTensor())

In [31]:
test_dataset = torchvision.datasets.MNIST(root = './data',train = False,transform = transforms.ToTensor())


In [32]:
train_dataloader = torch.utils.data.DataLoader(dataset = train_dataset,batch_size = 100, shuffle = True)

In [33]:
test_dataloader = torch.utils.data.DataLoader(dataset = test_dataset,batch_size = 100)

In [34]:
examples = iter(train_dataloader)
samples, labels = examples.next()

In [35]:
print(samples.shape,labels.shape)

torch.Size([100, 1, 28, 28]) torch.Size([100])


## implementing Neural Network for Multiple Classes

In [37]:
# same as above just sigmoid won't be used

In [38]:
class Neural_Network2(nn.Module):
    def __init__(self,input_size,hidden_size , num_classes):
        super(Neural_Network2, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [39]:
model = Neural_Network2(784,100,10)

## Loss

In [40]:
loss = nn.CrossEntropyLoss()

## Optimizer

In [41]:
optimizer = torch.optim.Adam(model.parameters(),lr= 0.01)

## Writing to Tensorboard

In [46]:
writer.add_graph(model,samples.reshape(-1,28*28))
writer.close()

## Condition of training

In [48]:
num_epochs = 10

## Initializing variables for Tensorboard

In [49]:
running_loss = 0.0
running_correct = 0.0

## Training

In [51]:

for epoch in range(1,num_epochs+1):
    for i , (images, labels) in enumerate(train_dataloader):
        ## reshape images
        images = torch.reshape(images, (-1, 28*28))
        y_pred = model(images)
        l = loss(y_pred,labels)
        
        ## =========================Variables for TensorBoard Updated======================
        running_loss += l.item()
        _,predicted = torch.max(y_pred.data,1)
        running_correct += (predicted == labels).sum().item()
        ## ========================Variables ended=============================
        
        
        l.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (i+1)%100==0:
            print(f"[INFO]:  epoch : {epoch}/{num_epochs}, loss: {l.item():.4f}")
            writer.add_scalar('training_loss',running_loss/100,epoch*len(train_dataloader)*i)
            writer.add_scalar('training_accuracy',running_correct/100,epoch*len(train_dataloader)*i)
            running_loss = 0.0
            running_correct = 0.0
    

[INFO]:  epoch : 1/10, loss: 0.4457
[INFO]:  epoch : 1/10, loss: 0.1645
[INFO]:  epoch : 1/10, loss: 0.1280
[INFO]:  epoch : 1/10, loss: 0.1694
[INFO]:  epoch : 1/10, loss: 0.2081
[INFO]:  epoch : 1/10, loss: 0.1296
[INFO]:  epoch : 2/10, loss: 0.1403
[INFO]:  epoch : 2/10, loss: 0.0118
[INFO]:  epoch : 2/10, loss: 0.0539
[INFO]:  epoch : 2/10, loss: 0.0509
[INFO]:  epoch : 2/10, loss: 0.0380
[INFO]:  epoch : 2/10, loss: 0.1460
[INFO]:  epoch : 3/10, loss: 0.1088
[INFO]:  epoch : 3/10, loss: 0.0570
[INFO]:  epoch : 3/10, loss: 0.0442
[INFO]:  epoch : 3/10, loss: 0.0896
[INFO]:  epoch : 3/10, loss: 0.1555
[INFO]:  epoch : 3/10, loss: 0.0718
[INFO]:  epoch : 4/10, loss: 0.0741
[INFO]:  epoch : 4/10, loss: 0.1441
[INFO]:  epoch : 4/10, loss: 0.0365
[INFO]:  epoch : 4/10, loss: 0.1610
[INFO]:  epoch : 4/10, loss: 0.1175
[INFO]:  epoch : 4/10, loss: 0.0283
[INFO]:  epoch : 5/10, loss: 0.0317
[INFO]:  epoch : 5/10, loss: 0.1242
[INFO]:  epoch : 5/10, loss: 0.0855
[INFO]:  epoch : 5/10, loss:

## test

In [27]:
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in test_dataloader:
        images = torch.reshape(images,(-1,784))
        outputs = model(images)
        
        _, predictions = torch.max(outputs,1)
        total += labels.shape[0]
        correct += (predictions==labels).sum().item()
        
    acc = 100.0*correct/total
    print(f"Accuracy : {acc:.2f}")
        
        
        

Accuracy : 96.76
